# Dataset

1. load csv file (panda, numpy)
2. split dataset. Example code:()
   ```
   random.shuffle(data) # change if you are using pandas dataframe
   training = data[:int(len(data)*0.8)]
   test = data[int(len(data)*0.8):]

   fold5 = KFold(5) # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
   for train_idx, val_idx in fold5.split(training):
      sub_val = training[val_idx]
      sub_train = training[train_idx]
      clf = model(sub_train, sub_val, ...) # training the model, and evaluate it on validation dataset
      performance(clf, test) # test the model on test dataset
   ```

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

#Naive bayes

1. model learning:

   Note:

   features: remove attributes that is not related to word (the last four attributes)

   labels: the last column

   count P(c) -> how many samples are positive, and how many are negtive

   if freq_word>0, then this word exists. You could use this to calculate P(a|c) -> for each class, what is the prob of each word

   remember to use laplace smoothing.

2. model evaluation (on val dataset -> performance(model, val)):
   
   for each new sample, $\prod{P(a|c)}P(c)$ if word is in the email(freq_word > 0); and find the maximum class
   

   

# KNN
1. model learning: None

2. model evaluation(on val dataset): You could use each row(exclude the last column) as the feature of the email. You do not have to recalcuate the freqency.

   ```
   Note:
   parallel programing
   numpy.cos() to calcuate the similarity
   ```

# LR

1. model learning: You could use each row(exclude the last column) as the feature of the email. You do not have to recalcuate the freqency.
    
    $y = sigmoid(MX)$

step 1: add one more column (all value is 1) in X -> X' = np.c_[np.ones((len(X), 1)), X]

step 2:vector M = np.random.randn(len(X[0])+1, 1);

key formula for step 3 (Note: n is the size of the TRAINING dataset; $cdot$ is dot production ):

1. $pred_y = sigmoid(M\cdot X')$

2. $loss = -\sum(y\cdot log(pred_y)+(1-y)\cdot log(1-pred_y))/n$

3. $gm=X'\cdot (pred_y - y)*2/n$

Step 3 example code:
   ```
   #Step 3: performing gradient descent on whole dataset:
   best_model = M
   best_performace = 0
   for i in range(epoch):
     pred_y = ...
     gm = ...
     _p = performace(model, val)
     if _p > best_performance:
        best_model = M
        best_performance = _p
     M = M - learning_rate*gm
   ```

2. model evaluation(on val dataset):
  
   calculate pred_y, if more than 0.5, then the predicted label is 1.

# Model Evaluation

https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
file = pd.read_csv("spambase.csv")

trainData, testData = train_test_split(file, test_size=0.2, random_state=19)

fTrn = trainData.iloc[:, :-1].values
sTrn = trainData.iloc[:, -1].values
fTest = testData.iloc[:, :-1].values
sTest = testData.iloc[:, -1].values

class NaiveBayes:

  def __init__(self):
    self.prProb = None
    self.cndProb = None

  def learn(self, xVal, yVal):
    self.prProb = {}
    cType, index = np.unique(yVal, return_counts=True)
    samp = len(yVal)
    for types, count in zip(cType, index):
      self.prProb[types] = count / samp

      self.cndProb = {}
      for types in cType:
        data = xVal[yVal == types]
        wordCount = np.sum(data > 0, axis=0) + 1
        self.cndProb[types] = wordCount / (len(data) + 2)

  def predict(self, val):
    prediction = []
    for data in val:
      posts = {}
      for types in self.prProb:
        outcome = np.prod(self.cndProb[types][data > 0])
        post = outcome * self.prProb[types]
        posts[types] = post
      prediction.append(max(posts, key=posts.get))
    return prediction
'''
class logisticRegression:

  def __init__(self, rate=0.01, iterations=1000):
    self.rate = rate
    self.iterations = iterations
    self.bias = None
    self.weight = None

  def learn(self, xVal, yVal):
    feature, sample = xVal.shape
    self.weight = np.zeros(feature)
    self.bias = 0

    for _ in range(self.iterations):
      lineMod = np.dot(self.weight, xVal) + self.bias
      prediction = 1/(1 + np.exp(lineMod))
      gradientW = (1/ sample) * np.dot(xVal.T, (prediction - yVal))
      gradientB = (1/ sample) * np.sum(prediction - yVal)

      self.weight -= self.rate * gradientW
      self.bias -= self.rate * gradientB

  def predict(self, value):
    lineMod = np.dot(value, self.weight) + self.bias
    prediction = 1/(1 + np.exp(-lineMod))

    types = []
    for prob in prediction:
      if prob > 0.5:
        types.append(1)
      else:
        types.append(0)
    return types
'''
class KNN:

  def __init__(self, value = 3):
    self.value = value


  def knn(self, xVal, yVal):
    self.xVal = xVal
    self.yVal = yVal

  def predict( self, xVal):
    yVal = []

    for val in xVal:
      sample = self.learn(val)
      yVal.append(sample)

    return np.array(yVal)

  def learn( self, val):
    distances = []

    for train in self.xVal:
      distance = np.sqrt(np.sum((val - train) ** 2))
      distances.append(distance)

    index = np.argsort(distance)[:self.value]

    kLabel = []

    for num in index:
      label = self.yVal[num]
      kLabel.append(label)

    comLable = np.bincount(kLabel).argmax()

    return comLable



navB = NaiveBayes()
navB.learn(fTrn, sTrn)
prediction = navB.predict(fTest)
'''
logReg = logisticRegression()
logReg.learn(fTrn, sTrn)
logRegPred = logReg.predict(fTest)
'''
KNN = KNN()
KNN.knn(fTrn, sTrn)
KNNPred = KNN.predict(fTest)

accuracy = accuracy_score(sTest, prediction)
confMatrix = confusion_matrix(sTest, prediction)
truePos = confMatrix[1,1]
falsePos = confMatrix[0,1]
AUC = roc_auc_score(sTest, navB.predict(fTest))
'''
logRegAcc = accuracy_score(sTest, logRegPred)
logRegConfMatrix = confusion_matrix(sTest, logRegPred)
logRegTruePos = logRegConfMatrix[1,1]
logRegFalsePos = logRegConfMatrix[0,1]
logRegAUC = roc_auc_score(sTest, logReg.predict(fTest))
'''
knnAcc = accuracy_score(sTest, KNNPred)
knnConfMatrix = confusion_matrix(sTest, KNNPred)
knnTruePos = knnConfMatrix[1,1]
knnFalsePos = knnConfMatrix[0,1]
knnAUC = roc_auc_score(sTest, KNN.predict(fTest))


print("Naive Bayes:")
print("Accuracy:", accuracy)
print("True Positives:", truePos)
print("False Positives:", falsePos)
print("Area Under Curve:", AUC)
print("-------------------------------")
print()
'''print("Logistic Regression:")
print("Accuracy:", logRegAcc)
print("True Positives:", logRegTruePos)
print("False Positives:", logRegFalsePos)
print("Area Under Curve:", logRegAUC)
print("-------------------------------")
print()'''
print("KNN:")
print("Accuracy:", knnAcc)
print("True Positives:", knnTruePos)
print("False Positives:", knnFalsePos)
print("Area Under Curve:", knnAUC)


Naive Bayes:
Accuracy: 0.8197611292073833
True Positives: 362
False Positives: 157
Area Under Curve: 0.8451433472188189
-------------------------------

KNN:
Accuracy: 0.5971769815418024
True Positives: 0
False Positives: 0
Area Under Curve: 0.5
